In [10]:
# Imports
import json
import numpy as np
import plotly.plotly as py
import matplotlib as mpl
mpl.use('Agg')
from matplotlib import cycler
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
import os
import glob2
import re
import h5py

from IPython.display import display

<b> Using saved local map data...

### ISSUE 1:

Why are bias_x and bias_y changing while bee is scenting? Should emit based on initial calculation before first timestep of scenting. 

<B> FIX: Upon activation when threshold is met, bee stands still and emits in 1 direction for the (6) timesteps of scenting. Indented for line 181 in Bees.py.

In [3]:
experiment_dir = "experiments/07M_17D-20H_45M_11S_Q0.15_W0.15_D0.25_T0.001_wb1/experiment0"

In [4]:
with open(experiment_dir + "/data/measurements.json", "r") as f:
    measurements = json.load(f)

In [16]:
# While worker is scenting; position is constant during these 6 timesteps.
measurements['position_history']['worker_2'][4:4+6]

[{'bias_x': 0.0,
  'bias_y': -0.9999999989999999,
  'found_queen_direction': True,
  'local_map': [[0.0020428671224579057,
    0.0022588464730266206,
    0.0024951635354296627],
   [0.0020438888114149936, 0.0022599761786603733, 0.0024964114291385845],
   [0.0020428671224579325, 0.0022588464730266505, 0.002495163535429696]],
  'scenting': True,
  'x': 0.0,
  'y': -1.0},
 {'bias_x': 0.0,
  'bias_y': 0.9999999989999999,
  'found_queen_direction': True,
  'local_map': [[0.6534640544532948, 0.642321652443168, 0.6289194741759708],
   [0.6547641715046739, 0.6435987826789523, 0.6301690442404114],
   [0.653464054453328, 0.6423216524432007, 0.6289194741760028]],
  'scenting': True,
  'x': 0.0,
  'y': -1.0},
 {'bias_x': 0.0,
  'bias_y': -0.9999999989999999,
  'found_queen_direction': True,
  'local_map': [[1.0691031195796403, 1.0753687116802664, 1.0786948882324097],
   [1.0707905713878327, 1.0770758880433065, 1.0804165296709212],
   [1.0691031195796834, 1.07536871168031, 1.0786948882324534]],
  '

<b> See if FIX worked: Yes.

In [75]:
experiment_dir2 = "experiments/moving_07M_18D-16H_21M_31S_Q0.15_W0.15_D0.25_T0.001_wb1/experiment0"
with open(experiment_dir2 + "/data/measurements.json", "r") as f:
    measurements2 = json.load(f)
    
# While worker is scenting; position is constant during these 6 timesteps.
worker = "worker_4"
for t_i, t in enumerate(measurements2['position_history'][worker]):
    if t["scenting"]:
        print("TIMESTEP {}... {} \n".format(t_i, t))

TIMESTEP 4... {'x': -0.95, 'y': -0.05, 'found_queen_direction': True, 'scenting': True, 'bias_x': -0.7071067806865474, 'bias_y': -0.7071067806865474, 'local_map': [[0.0032849593286873964, 0.0033030763811287794, 0.0033179737186395434], [0.0036141412317188034, 0.003634073772634546, 0.0036504639547808794], [0.003972335742761969, 0.003994243781116634, 0.004012258380490575]]} 

TIMESTEP 5... {'x': -0.95, 'y': -0.05, 'found_queen_direction': True, 'scenting': True, 'bias_x': -0.7071067806865474, 'bias_y': -0.7071067806865474, 'local_map': [[0.0032849593286873964, 0.0033030763811287794, 0.0033179737186395434], [0.0036141412317188034, 0.003634073772634546, 0.0036504639547808794], [0.003972335742761969, 0.003994243781116634, 0.004012258380490575]]} 

TIMESTEP 6... {'x': -0.95, 'y': -0.05, 'found_queen_direction': True, 'scenting': True, 'bias_x': -0.7071067806865474, 'bias_y': -0.7071067806865474, 'local_map': [[0.0032849593286873964, 0.0033030763811287794, 0.0033179737186395434], [0.0036141412

### ISSUE 2:

Adjusted indices not correct. (Mysteriously, still works sometimes. Why?)

<B> FIX: New mapping between array indices and theoretical map with bounds [-1,1] 

--- With old adjusted indices:

In [78]:
# Initial local map that activated scenting (when x,y of worker > T)
local_map = measurements2['position_history']['worker_4'][4]['local_map']
local_map = np.array(local_map)
local_map

array([[0.00328496, 0.00330308, 0.00331797],
       [0.00361414, 0.00363407, 0.00365046],
       [0.00397234, 0.00399424, 0.00401226]])

In [79]:
# Concentration @ where bee is
current_c = local_map[1][1]
current_c

0.003634073772634546

Shown in array above, max index (before adjusted) is...

In [103]:
max_concentration = np.max(local_map[np.where(local_map > current_c)])
max_concentration_indices = list(np.where(local_map == max_concentration))

max_concentration, max_concentration_indices

(0.004012258380490575, [array([2]), array([2])])

Before, we adjusted indices this way below, which is wrong. 

Wrong adjusted indices: 1,1. Wrong scenting direction: -1,-1

Correct max direction: 1, -1. Correct scenting direction: -1,1

In [131]:
adjusted_indices_orig = [int(i)-1 for i in max_concentration_indices]

directions_to_queen_orig = { "x" : adjusted_indices_orig[0], "y": adjusted_indices_orig[1] }
directions_to_queen_orig

{'x': 1, 'y': 1}

Now, make correct mapping between array indices and array bounded by [-1,1]

In [132]:
max_concentration_indices_tup = tuple([int(index) for index in max_concentration_indices])
max_concentration_indices_tup

(2, 2)

In [133]:
# Dictionary of original indices : adjusted indices
mapping_dict = {
                (0,0): (-1,1),
                (1,0): (-1,0),
                (2,0): (-1,-1),
                (0,1): (0,1),
                (1,1): (0,0),
                (2,1): (0,-1),
                (0,2): (1,1),
                (1,2): (1, 0),
                (2,2): (1,-1),
               }

In [138]:
adjusted_indices = (0,0)

if max_concentration_indices_tup in mapping_dict.keys():
    adjusted_indices =  mapping_dict[max_concentration_indices_tup]
    print(adjusted_indices)

(1, -1)


Now see if this prints the correct direction for scenting... Yes. 

In [140]:
directions_to_queen = { "x" : adjusted_indices[0], "y": adjusted_indices[1] }
directions_to_queen

{'x': 1, 'y': -1}

In [141]:
if (directions_to_queen["x"] > 0):
    bias_direction_x = -1
elif (directions_to_queen["x"] == 0):
    bias_direction_x = 0
else:
    bias_direction_x = 1

if (directions_to_queen["y"] > 0):
    bias_direction_y = -1
elif (directions_to_queen["y"] == 0):
    bias_direction_y = 0
else:
    bias_direction_y = 1

In [142]:
bias_direction_x, bias_direction_y

(-1, 1)

In [143]:
magn = np.sqrt(bias_direction_x**2 + bias_direction_y**2) + 1e-9
magn

1.4142135633730952

In [144]:
w_b = 1

# Update bias - unit vectors
bias_x = w_b * (bias_direction_x / float(magn))
bias_y = w_b * (bias_direction_y / float(magn))

In [145]:
bias_x, bias_y

(-0.7071067806865474, 0.7071067806865474)

In [149]:
test_array = np.random.rand(3,3)
test_array

array([[0.01083315, 0.8836065 , 0.83887648],
       [0.94978114, 0.69694358, 0.14110976],
       [0.86154208, 0.08327307, 0.36780014]])

In [152]:
test_array[2,1]

0.08327306824555747

<b> Michael's attempt at fixing the adjusted indices... Not sure if it works!

In [96]:
mapper = np.zeros((3, 3, 2))
mapper[0,:,1] = 1
mapper[1,:,1] = 0
mapper[2,:,1] = -1

mapper[:,0,0] = -1
mapper[:,1,0] = 0
mapper[:,2,0] = 1

In [133]:
x = np.random.random((3, 3))
x[1,1] = 0
x

array([[0.23998572, 0.20021564, 0.72147306],
       [0.42808672, 0.        , 0.1854638 ],
       [0.21078409, 0.7930467 , 0.42853327]])

In [134]:
idx = np.unravel_index(x.argmax(), x.shape)

In [135]:
idx

(2, 1)

In [141]:
vector_to_queen = mapper[idx]
vector_to_queen

array([ 0., -1.])

In [137]:
direction_away_from_queen = -mapper[idx]
direction_away_from_queen

array([-0.,  1.])

In [142]:
np.linalg.norm(direction_away_from_queen)

1.0

In [138]:
np.sqrt(direction_away_from_queen[0]**2 + direction_away_from_queen[1]**2)

1.0

In [139]:
direction_away_from_queen[0]/1.0

-0.0

In [140]:
direction_away_from_queen[1]/1.0

1.0

In [84]:
np.array(idx)

array([-1,  2])

In [79]:
max_c = x[idx]
max_c

0.9798661659343952

In [80]:
idx_a = [int(i)-1 for i in idx]
idx_a

[-1, 0]

In [71]:
directions_to_queen = { "x" : idx_a[1], "y": idx_a[0] }
directions_to_queen

{'x': 0, 'y': -1}

In [72]:
if (directions_to_queen["x"] > 0):
    bias_direction_x = 1
elif (directions_to_queen["x"] == 0):
    bias_direction_x = 0
else:
    bias_direction_x = -1

if (directions_to_queen["y"] > 0):
    bias_direction_y = 1
elif (directions_to_queen["y"] == 0):
    bias_direction_y = 0
else:
    bias_direction_y = -1
bias_direction_x, bias_direction_y

(0, -1)

In [73]:
zz = [int(i)+1 for i in [bias_direction_x, bias_direction_y]]
zz

[1, 0]

In [69]:
x[2, 1]

0.47050230394699133

In [76]:
np.sqrt(zz[0]**2 + zz[1]**2)

1.0

In [74]:
np.linalg.norm(zz)

1.0